# **예제 2.1 토큰화 코드**

> 텍스트를 숫자 아이디로 바꾸는 과정
> 단어 단위(띄어쓰기 단위)로 토큰화 수행

In [1]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list : ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx : ", str2idx)
print("idx2str : ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids : ", input_ids)

input_text_list :  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx :  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str :  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids :  [0, 1, 2, 3, 4]


# **예제 2.2 토큰 아이디에서 벡터로 변환**
> 토큰의 의미를 담기 위해 벡터(숫자 집합)으로 변환

In [2]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5,16)
input_embeddings = input_embeddings.unsqueeze(0) # (1,5,16)
input_embeddings.shape

torch.Size([1, 5, 16])

# **예제 2.3 절대적 위치 인코딩**
> 새로운 임베딩 층을 추가 -> 위치 인덱스에 따라 임베딩을 더함

In [3]:
embedding_dim = 16
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids)) # (5,16)
token_embeddings = token_embeddings.unsqueeze(0) # (1,5,16)
token_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

# **예제 2.4 쿼리, 키, 값 벡터 만드는 nn.Linear 층**
> 가중치 -> nn.Linear 층 사용       
> 각 가중치 생성 -> weight_q, weight_k, weight_v     
> 입력 → input_embedding

⇒ 선형 층에 통과시켜 쿼리, 키, 값 생성

In [4]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위하 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
querys = weight_q(input_embeddings) # (1,5,16)
keys = weight_k(input_embeddings) # (1,5,16)
values = weight_v(input_embeddings) # (1,5,16)

# **예제 2.5 스케일 점곱 방식의 어텐션**
> 단어와의 관계를 얼마나 반영할지 명확하게 정하기 어렵기에 합을 1로 만들 수 있도록 소프트 맥스 취하기

In [5]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
	weights = F.softmax(scores, dim=-1)
	return weights @ values

# **예제 2.6 어텐션 연산의 입력과 출력**
> 어텐션을 거치고 나면 입력과 형태는 동일하면서 주변 토큰과의 관련도에 따라 값 벡터를 조합한 새로운 토큰 임베딩 생성

In [6]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)

# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


# **예제 2.7 어텐션 연산을 수행하는 AttentionHead 클래스**
> __init__ 메서드 -> 선형 층(weight_q, weight_k, weight_v) 생성   
> forward 메서드 -> 선형 층을 통해 쿼리, 키, 값 벡터 생성    
> compute_attention 함수 사용 -> 어텐션 연산 수행

In [7]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

# **예제 2.8 멀티 헤드 어텐션 구현**
> AttentionHead와 대부분의 코드가 동일하나

1. 헤드 수만큼 연산을 수행하기 위해 쿼리, 키, 값을 n_head개로 쪼개기 -> 여러 선형 층
1. 각각의 어텐션 계산 -> h번의 스케일 점곱 어텐션
2. 입력과 같은 형태로 다시 변환 -> 어텐션 결과 연결
3. 마지막으로 선형층 통과시키고 최종 결과 반환 -> 마지막 선형 층


In [8]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    attention = compute_attention(querys, keys, values, self.is_causal)
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

# **예제 2.9 층 정규화 코드**
> 파이토치가 제공하는 LayerNorm 클래스 이용     
> -> nn.LayerNorm 클래스로 층 정규화 레이스 만들기    
> (토큰 임베딩 차원(embedding_dim)인자로 전달, 입력 임베딩을 층 정규화 레이어에 통과시켜 정규화된 임베딩(norm_x)로 만들기)

In [9]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1,5,16])

norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data # 실제로 평균과 표준편차 확인하기

# (tensor([[7.4506e-09, 0.0000e+00, 1.1176e-08, 2.9802e-08, 1.1176e-08]]),
# tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[-1.8626e-08,  0.0000e+00,  2.2352e-08, -2.2352e-08, -1.6764e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

# **예제 2.10 피드 포워드 층 코드**
>**구성**
- 선형 층
- 드롭아웃 층
- 층 정규화
- 활성 함수

In [18]:
class PreLayerNormFeedForward(nn.Module) :
	def __init_(self, d_model, dim_feedforward, dropout):
		super().__init_()
		self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
		self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
		self.dropout1 = nn.Dropout(dropout) # 드롭아웃 층 1
		self.dropout2 = nn.Dropout(dropout) # 드롭아웃 층 2
		self.activation = nn.GELU() # 활성 함수
		self.norm = nn.LayerNorm(d_model) # 층 정규화

def forward(self, src):
  x = self.norm(src)
  x = x + self. linear2(self.dropoutl(self.activation(self.linear1(x))))
  x = self.dropout2(x)
  return x

# **예제 2.11 인코더 층**
- 입력인 src를 self.norm1을 통해 층 정규화
- 멀티 헤드 어텐션 클래스를 인스턴스화한 self.attn을 통해 멀티 헤드 어텐션 연산 수행
- 잔차 연결을 위해 어텐션 결과에 드롭아웃을 취한 self.dropout1(attn_output)과 입력(src)을 더해준다
- self.feed_forward(x)를 통해 피드 포워드 연산

In [11]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.linear1 = nn.Linear(d_model)
    self.dropout = nn.Dropout(dropout)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

def forward(self, src):
  norm_x = self.norm1(src)
  attrn_output = self.attn(norm_x, norm_x, norm_x)
  x = src + self.dropout1(attn_output)

  # 피드 포워드
  x = self.feed_forward(x)
  return x

# **예제 2.12 인코더 구현**


In [12]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
      output = mod(output)
    return output

# **예제 2.13 디코더에서 어텐션 연산(마스크 어텐션)**


In [19]:
def compute_attention(querys, keys, values, is_causal=False):
  dim_k = querys.size(-1) # 16
  scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1,5,5)
  if is_causal:
    query_length = querys.size(-2)
    key_length = keys.size(-2)
    temp_mask = torch.ones(query_length, key_length,
                           dtype=torch.bool).trill(diagonal=0)
    scores = scores.masked_fill(temp_mask == False, float("-inf"))
  weights = F.softmax(scores, dim=-1) # (1,5,5)
  return weigths @ values # (1,5,16)

# **예제 2.14 크로스 어텐션이 포함된 디코더 층**